# Demo of a Tesla customer support assistant chatbot

In [1]:
import os
from typing import Tuple, List, Mapping, Text, Any
import torch

# simple hack to support import module from parent directory
import sys
sys.path.append('../')

from rag_llama.core.retrievers import RerankRetriever
from rag_llama.core.generation import Llama, Dialog


## Define system message and input query format templates. 
Note this are for single-turn chat, similar to how search works.

In [3]:
SYSTEM_MESSAGE = """
You are an assistant to a Tesla customer support team. Your job is to answer customer's questions to the best of your ability.
"""

SYSTEM_MESSAGE_RAG = SYSTEM_MESSAGE + """
 Your role involves leveraging a set of reference documents to ensure accurate responses. 
 While some documents may not directly apply to every question, focus solely on those that seem pertinent. 
 Avoid referencing or citing documents not provided. 
 Craft concise answers, incorporating relevant sections from the provided documents to assist customers effectively.
"""


def get_formatted_input_dialog_for_hyde(query: str) -> Dialog:
    formatted_query = f'Please write a short passage to answer the question to the best of your ability. The passage should not exceed 200 words.\nQuestion: {query}\n\nPassage: '
    dialog = [{'role': 'system', 'content': ""},{'role': 'user', 'content': formatted_query}]
    return dialog


def get_formatted_input_dialog(query: str, doc_strs: str=None) -> Dialog:

    if doc_strs is not None and len(doc_strs) > 10:
        formatted_query = f"Question:\n{query}\n\n----\n\nDocuments:\n{doc_strs}"
        dialog = [{'role': 'system', 'content': SYSTEM_MESSAGE_RAG}, {'role': 'user', 'content': formatted_query}]
    else:
        dialog = [{'role': 'system', 'content': SYSTEM_MESSAGE},{'role': 'user', 'content': query}]
    
    return dialog



Create the reranking retrieval instance and the LLaMA 2 chat generator instance.

In [4]:
doc_embed_file = "../data/Tesla_manual_embeddings.pk"
llama_model_ckpt = os.path.expanduser("~/models/meta_llama2/llama-2-7b-chat/consolidated.pth")
llama_tokenizer_ckpt = os.path.expanduser("~/models/meta_llama2/tokenizer.model")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# LLM parameters
max_seq_len = 4096
max_gen_len = 512
temperature = 0.4 # model sampling temperature
top_p = 0.7 # model sampling top P

# RAG specific parameters
top_k = 50 # select top K items to retrieve during naive retrieval
top_n = 4 # select top N items based on scores from ranking model

In [5]:
rerank_retriever = RerankRetriever(embed_file=doc_embed_file, device=device)

generator = Llama.build(
    ckpt_path=llama_model_ckpt,
    tokenizer_path=llama_tokenizer_ckpt,
    max_seq_len=max_seq_len,
    max_batch_size=1,
    device=device,
)

Loading sentence-transformers/all-MiniLM-L6-v2 model and tokenizer from HuggingFace...
Loading cross-encoder/ms-marco-MiniLM-L-6-v2 model and tokenizer from HuggingFace...
Starting to load tokenizer checkpoint '/home/michael/models/meta_llama2/tokenizer.model' ...
Starting to load model checkpoints '/home/michael/models/meta_llama2/llama-2-7b-chat/consolidated.pth' ...
Model checkpoint loaded in 16.95 seconds


## Main logic for retrieval and LLM generation

In [6]:

def generate_hyde_response(query: str) -> str:
    dialog = get_formatted_input_dialog_for_hyde(query)
    result = generator.chat_completion(
        [dialog],  # input needs to be a batch of dialogs
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

    response = result[0]['generation']['content']
    print(f'HyDE response:\n{response}')
    return response


def get_formatted_text_from_documents(documents: List[Mapping[Text, Any]]) -> str:
    """Join multiple documents into a single document string"""
    result = "\n\n".join([item['formatted_text'] for item in documents])
    return result


def run_chat_completions(query: str, use_hyde: bool=True) -> Tuple[str]:
    """Run chat completion with the same query multiples times, with and without RAG"""

    # build a list of dialogs for cases with and without RAG
    # case 1 - default without RAG
    dialogs = [get_formatted_input_dialog(query, None)]

    # case 2 - with RAG
    retrieved_items = rerank_retriever.retrieve(query, top_k, top_n)
    ref_doc_strs = get_formatted_text_from_documents(retrieved_items)
    dialogs.append(get_formatted_input_dialog(query, ref_doc_strs))

    # case 3 - finally RAG with HyDE
    if use_hyde:
        hyde_response = generate_hyde_response(query)
        retrieved_items_hyde = rerank_retriever.retrieve(hyde_response, top_k, top_n)
        ref_doc_strs_hyde = get_formatted_text_from_documents(retrieved_items_hyde)
        dialogs.append(get_formatted_input_dialog(query, ref_doc_strs_hyde))
    
    # passing it separately is much faster due to the fact dialog without RAG is much shorter
    results = []
    for dialog in dialogs:
        result = generator.chat_completion(
            [dialog],  # input needs to be a batch of dialogs
            max_gen_len=max_gen_len,
            temperature=temperature,
            top_p=top_p,
        )

        results.extend(result)

    response = results[0]['generation']['content']
    response_with_rag = results[1]['generation']['content']
    # add reference documents at the end of response
    response_with_rag += f"\n\nReference documents:\n{ref_doc_strs}"

    response_with_rag_hyde = ''
    if use_hyde:
        response_with_rag_hyde = results[2]['generation']['content']
         # add reference documents at the end of response
        response_with_rag_hyde += f"\n\nReference documents:\n{ref_doc_strs_hyde}"

    return response, response_with_rag, response_with_rag_hyde


A simple hack to display the chat completions with and without RAG side-by-side for better comparison

In [7]:
from IPython.display import HTML, display

def display_completions_in_columns(query: str, response: str, response_with_rag: str, response_with_rag_and_hyde: str) -> None:
    """
    Display content in columns side-by-side.
    
    Parameters:
    query (str): user query
    response (str): response for left column.
    response_with_rag (str): response for middle column.
    response_with_rag_and_hyde (str): response for right column.
    """

    # Convert newlines to HTML line breaks and bulleted lists to HTML list items
    def handle_newline(input_strings) -> str:
        if input_strings is None:
            return
        return input_strings.replace('\n', '<br>')
    
    response = handle_newline(response)
    response_with_rag = handle_newline(response_with_rag)
    response_with_rag_and_hyde = handle_newline(response_with_rag_and_hyde)

    html_content = f'''
    <div>
        <div style="padding: 20px 0; font-weight: bold;">User: {query}</div>
        <div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px;">
            <div>
                <div style="font-weight: bold;">Assistant:</div>
                {response}
            </div>
            <div>
                <div style="font-weight: bold;">Assistant with RAG:</div>
                {response_with_rag}
            </div>
            <div>
                <div style="font-weight: bold;">Assistant with RAG and HyDE:</div>
                {response_with_rag_and_hyde}
            </div>
        </div>
    <div>
    '''
    display(HTML(html_content))

In [8]:
def ask_question(query: str, use_hyde: bool=True):
    response, response_with_rag, response_with_rag_and_hyde = run_chat_completions(query, use_hyde)
    display_completions_in_columns(query, response, response_with_rag, response_with_rag_and_hyde)

## Now we can start asking questions about Tesla cars

The model with go over the same query multiple times, with and without using RAG. The responses will be displayed in different columns for better comparisons.

In [9]:
ask_question('How to enable Autopilot on Tesla Model S 2018 model?')

HyDE response:
 To enable Autopilot on a 2018 Tesla Model S, follow these steps:
1. Ensure your vehicle is in Autopilot-enabled mode by checking the Autopilot icon on the instrument panel.
2. Press the Autopilot button on the steering wheel to activate the system.
3. Once activated, the vehicle will begin to perform various maneuvers, such as steering, accelerating, and braking, to maintain its lane position and speed.
4. To adjust the Autopilot settings, use the touchscreen display to navigate to the Autopilot menu.
5. From here, you can adjust the system's sensitivity, enable or disable certain features, and view the vehicle's Autopilot logs.
6. To exit Autopilot mode, press the Autopilot button again or tap the brake pedal.
By following these steps, you can safely and easily enable Autopilot on your 2018 Tesla Model S. It's important to familiarize yourself with the system's features and limitations to ensure a safe and enjoyable driving experience.


In [10]:
ask_question('Under what circumstances that I should not use full self-driving on my Tesla car?')

HyDE response:
 When using a Tesla with full self-driving capabilities, there are certain circumstances where it is best to exercise caution and not rely solely on the autonomous driving system. These include:

* Construction zones or road work: Full self-driving may not be able to handle unexpected obstacles or lane shifts in construction zones, so it's best to take control of the vehicle in these areas.
* Inclement weather: Full self-driving may struggle with heavy rain, snow, or fog, so it's important to be prepared to take control of the vehicle in these conditions.
* Low visibility: If visibility is poor due to weather or other factors, it's best to take control of the vehicle to ensure safe operation.
* Complex intersections: Full self-driving may have difficulty navigating complex intersections with multiple lanes or turns, so it's best to take control in these situations.
* Emergency situations: In the event of an emergency, such as a pedestrian stepping into the road, it's imp

In [11]:
ask_question('How to open the door of a Tesla Model S car when the power is very low?')

HyDE response:
 When the power is low in a Tesla Model S, opening the door can be a challenge. However, there are a few methods you can use to access the vehicle. Firstly, you can try using the emergency door release lever, which is located inside the vehicle near the brake pedal. This lever will open the door in case of an emergency or if the door is stuck. Secondly, you can use the remote unlock feature on the Tesla app or the key fob. Simply press and hold the unlock button on the app or fob, and the door will unlock. If the power is very low, you may need to use the tow truck or roadside assistance to open the door. It's important to prioritize safety when dealing with a low-power situation and seek professional help if necessary.


In [12]:
ask_question('Can I use autopilot in raining or snowing conditions?')

HyDE response:
 Sure, here's a passage that answers the question to the best of my ability:

"Using autopilot in raining or snowing conditions is generally not recommended. The sensors and cameras on an autonomous vehicle's system may have difficulty detecting and interpreting the road markings and obstacles in wet and slippery conditions, which can lead to errors in navigation and control. Additionally, the vehicle's tires may lose traction and hydroplaning may occur, which can further compromise the vehicle's ability to safely navigate the road. As a result, many autonomous vehicle manufacturers and regulatory agencies advise against using autopilot in rainy or snowy weather, and instead recommend manual driving until the conditions improve."


In [13]:
ask_question('What should I do if the touchscreen of my Tesla car is not responding?')

HyDE response:
 If the touchscreen of your Tesla car is not responding, there are several steps you can take to troubleshoot and resolve the issue. First, check to see if the display is frozen or if there is a software glitch. If the issue persists, try restarting the car by turning off the engine and then restarting it. If the touchscreen still does not respond, try using the physical controls on the steering wheel or center console. If the issue persists, you can also try resetting the car's system by pressing and holding the button located on the center console. If none of these steps work, you can contact a Tesla service center for further assistance. It is important to address any issues with the touchscreen as soon as possible to ensure the safety and proper functioning of your vehicle.


In [14]:
ask_question('Do I need to change oil regularly for my Tesla car?')

HyDE response:
 Yes, it is important to change the oil in your Tesla car regularly to ensure optimal performance and longevity of the vehicle. Unlike traditional gasoline-powered cars, Teslas do not have a traditional oil system and do not require oil changes as frequently. However, it is still important to check and maintain the electric vehicle's (EV) fluids, including the transmission fluid, coolant, and brake fluid. Tesla recommends checking these fluids every 12,000 to 15,000 miles and replacing them as needed. Additionally, it is important to keep the tires properly inflated and to rotate them regularly to ensure even wear and tear. By following these maintenance tips, you can help ensure your Tesla runs smoothly and efficiently for many years to come.


In [15]:
ask_question('Can I wash my Tesla car using regular car washing service?')

HyDE response:
 No, it's not recommended to wash your Tesla car using regular car washing services. Tesla cars have a special coating on their exteriors that helps to protect the paint and maintain its glossy finish. Regular car washes can scratch or damage this coating, which can lead to fading, oxidation, and other cosmetic issues. Tesla recommends using their own specialized car wash products and techniques to keep your car looking its best. These products are specifically designed to clean and protect the unique materials used in Tesla's vehicles, and they are gentler on the paint and other surfaces than regular car washes. By using the right products and techniques, you can help to maintain your Tesla's showroom appearance and ensure it stays in top condition for years to come.
